In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [5]:
import csv
with open("/home/aditi/Desktop/7th Sem/DWDM/document-information-retrieval/wikilink_ner.csv", "r") as f:
    reader = csv.reader(f)
    i = next(reader)
    print(i)

['Mention', ' DBpedia Uri', ' NIF Uri', ' Type']


In [6]:
csv_f = 'wikilink_ner.csv'
df = pd.read_csv(csv_f)

ParserError: Error tokenizing data. C error: Expected 4 fields in line 328356, saw 5


In [3]:
corpus=["She went to the airport to see him off.","I prefer reading to writing.","Los Angeles is in California. It's southeast of San Francisco.","I ate a burger then went to bed.","Compare your answer with Tom's.","I had hardly left home when it began to rain heavily.","If he had asked me, I would have given it to him." , "I could have come by auto, but who would pay the fare?","Whatever it may be, you should not have beaten him.","You should have told me yesterday","I should have joined this course last year.","Where are you going?","There are too many people here.","Everyone always asks me that.","I didn't think you were going to make it.",
        "Be quiet while I am speaking.","I can't figure out why he said so."] 
#Count Vectoriser then tidf transformer

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X)
print(tfidf.shape) 

(17, 93)


In [5]:
from sklearn.cluster import KMeans

num_clusters = 5 
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)
clusters = km.labels_.tolist()

In [7]:
import pandas as pd
idea={'Idea':corpus, 'Cluster':clusters} #Creating dict having doc with the corresponding cluster number.
frame=pd.DataFrame(idea,index=[clusters], columns=['Idea','Cluster']) # Converting it into a dataframe.

In [8]:
print("\n")
print(frame) #Print the doc with the labeled cluster number.
print("\n")
print(frame['Cluster'].value_counts()) #Print the counts of doc belonging to each cluster.



                                                Idea  Cluster
0            She went to the airport to see him off.        0
0                       I prefer reading to writing.        0
2  Los Angeles is in California. It's southeast o...        2
0                   I ate a burger then went to bed.        0
4                    Compare your answer with Tom's.        4
1  I had hardly left home when it began to rain h...        1
1  If he had asked me, I would have given it to him.        1
1  I could have come by auto, but who would pay t...        1
2  Whatever it may be, you should not have beaten...        2
2                  You should have told me yesterday        2
2        I should have joined this course last year.        2
3                               Where are you going?        3
3                    There are too many people here.        3
1                      Everyone always asks me that.        1
3          I didn't think you were going to make it.        3
2     